## N = 24

In [7]:
import numpy as np
import torch
import torch
import matplotlib.pyplot as plt

print(torch.cuda.is_available())


# load date
X = np.load("Datasets/kryptonite-30-X.npy")
y = np.load("Datasets/kryptonite-30-y.npy")
print(X.shape, y.shape)

features = X.shape[1]

label_dict = {}
data_dict = {}

for i in range(0, X.shape[0]):
    key = 0
    XX = []
    for j in range(0, features):
        if X[i][j] > 0.5:
            key += 2**j
            XX.append(X[i][j] - 1.0)
        else:
            XX.append(X[i][j])
    if key not in label_dict:
        label_dict[key] = []
        data_dict[key] = []
    label_dict[key].append(y[i])
    data_dict[key].append(XX)

flip_count = [0 for i in range(0, features)]
nflip_count = [0 for i in range(0, features)]

for key in label_dict:
    nw = max(label_dict[key], key=label_dict[key].count)
    for j in range(0, features):
        flip_key = key ^ 2**j
        if flip_key in label_dict:
            to = max(label_dict[flip_key], key=label_dict[flip_key].count)
            if nw != to:
                flip_count[j] += 1
            else:
                nflip_count[j] += 1

if_flip = [0 for i in range(0, features)]

for i in range(0, features):
    print(i, flip_count[i], nflip_count[i])
    if flip_count[i] > nflip_count[i]:
        if_flip[i] = 1

correct = 0
total = 0
origin_password = 0

# first_key = next(iter(label_dict))
# first_label = max(label_dict[first_key], key=label_dict[first_key].count)
# for i in range(0, features):
#     if first_key & 2**i:
#         origin_password ^= 2**i


for key in label_dict:
    predict = 1
    for i in range(0, features):
        if if_flip[i] and (key & 2**i):
            predict ^= 1
    if predict == max(label_dict[key], key=label_dict[key].count):
        correct += 1
    else:
        print(bin(key), predict, max(label_dict[key], key=label_dict[key].count))
    total += 1

print(correct, total, correct / total)
            

X = []
y = []

for key in label_dict:
    if (len(label_dict[key]) >= 1):
        label = max(label_dict[key], key=label_dict[key].count)
        for j in range(0, features):
            if key & 2**j:
                X.append(1)
            else:
                X.append(0)
        X.extend(data_dict[key][0])
        y.append(label)

X = np.array(X).reshape(-1, features * 2)
y = np.array(y)
print(X.shape, y.shape)

True
(60000, 30) (60000,)
0 0 2
1 2 0
2 0 0
3 2 0
4 2 0
5 10 0
6 0 4
7 0 2
8 4 2
9 2 0
10 6 0
11 2 0
12 0 2
13 0 2
14 0 4
15 2 0
16 10 0
17 0 2
18 4 0
19 0 10
20 0 4
21 2 0
22 6 0
23 2 0
24 0 8
25 2 0
26 0 0
27 4 0
28 4 0
29 6 0
30144 59999 0.5024083734728912
(59999, 60) (59999,)


In [8]:
import numpy as np
import torch
from tqdm import tqdm
from torch.utils.data import TensorDataset, DataLoader, random_split
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torchvision import models
from torch.optim.lr_scheduler import CosineAnnealingLR

# Convert numpy arrays to PyTorch tensors
X_tensor = torch.tensor(X, dtype=torch.float)
y_tensor = torch.tensor(y, dtype=torch.float)

# create a TensorDataset
dataset = TensorDataset(X_tensor, y_tensor)

# define split sizes (60% train, 20% validation, 20% test)
train_size = int(0.8 * len(dataset))
val_size = int(0.1 * len(dataset))
test_size = len(dataset) - train_size - val_size

# Split the dataset into train, validation, and test
train_dataset, val_dataset, test_dataset = random_split(dataset, [train_size, val_size, test_size])

# Create DataLoaders for each subset
batch_size = 512
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

# check data loader output
for X_batch, y_batch in tqdm(train_loader):
    # Reshape X_batch to have a single channel  # Add a channel dimension
    print(X_batch.shape, y_batch.shape)  # X_batch is of shape [batch_size, 1, 15]
    break

class MLP(nn.Module):
    def __init__(self, input_size=60, hidden_size=24, output_size=1):
        super(MLP, self).__init__()

        self.layer1 = nn.Linear(input_size, hidden_size * 2)
        self.layer2 = nn.Linear(hidden_size * 2, hidden_size)
        # self.layer3 = nn.Linear(hidden_size * 2, hidden_size)
        # self.layer4 = nn.Linear(hidden_size * 4, hidden_size * 2)
        # self.layer5 = nn.Linear(hidden_size * 2, hidden_size)
        self.layer6 = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        x = F.relu(self.layer1(x))
        x = F.relu(self.layer2(x))
        # x = F.relu(self.layer3(x))
        # x = F.relu(self.layer4(x))
        # x = F.relu(self.layer5(x))
        x = torch.sigmoid(self.layer6(x))  # sigmoid activation for binary output
        return x

# initialize the model, loss function, and optimizer
# Initialize the ResNet18 model
# model = CustomResNet18()
# criterion = nn.BCELoss()  # 使用二元交叉熵损失
# optimizer = torch.optim.Adam(model.parameters(), lr=0.001)


def train_epoch(model, train_loader, criterion, optimizer, scheduler, device):
    model.train()
    total_loss = 0
    correct = 0
    total = 0
    
    # 使用tqdm显示进度条
    # train_pbar = tqdm(train_loader, desc='Training')
    # for batch_idx, (inputs, targets) in enumerate(train_pbar):
    for batch_idx, (inputs, targets) in enumerate(train_loader):
        inputs, targets = inputs.to(device), targets.to(device)
        
        outputs = model(inputs).squeeze()

        predictions = torch.round(outputs)
        correct += torch.eq(predictions,targets).sum().item()
        total += len(targets)
        
        loss = criterion(outputs, targets)
        total_loss += loss.item()
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        
        # 计算准确率
        
        # 更新进度条信息
        # train_pbar.set_postfix({
        #     'loss': f'{total_loss/(batch_idx+1):.4f}',
        #     'acc': f'{100.*correct/total:.2f}%'
        # })
    
    scheduler.step()

    return total_loss / len(train_loader), correct / total

def validate(model, val_loader, criterion, device):
    model.eval()
    total_loss = 0
    correct = 0
    total = 0
    
    with torch.no_grad():
        # val_pbar = tqdm(val_loader, desc='Validation')
        # for batch_idx, (inputs, targets) in enumerate(val_pbar):
        for batch_idx, (inputs, targets) in enumerate(val_loader):
            inputs, targets = inputs.to(device), targets.to(device)
            
            outputs = model(inputs).squeeze()
            loss = criterion(outputs, targets)
            
            total_loss += loss.item()
            predictions = torch.round(outputs)
            correct +=  torch.eq(predictions, targets).sum().item()
            total += len(targets)
            
            # val_pbar.set_postfix({
            #     'loss': f'{total_loss/(batch_idx+1):.4f}',
            #     'acc': f'{100.*correct/total:.2f}%'
            # })
    
    return total_loss / len(val_loader), correct / total

torch.manual_seed(42)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = MLP().to(device)
criterion = nn.BCELoss()
optimizer = optim.Adam(model.parameters(), lr=1e-3, weight_decay=1e-4)
scheduler = CosineAnnealingLR(
    optimizer,
    T_max=100,  # 总epoch数
    eta_min=1e-5  # 最小学习率
)

epochs = 100
best_acc = 0
for epoch in range(epochs):
    # print(f'\nEpoch {epoch+1}/{epochs}')
    
    train_loss, train_acc = train_epoch(model, train_loader, criterion, optimizer, scheduler, device)
    val_loss, val_acc = validate(model, val_loader, criterion, device)
    
    if epoch % 10 == 0:
        print(f'Epoch {epoch+1}/{epochs}')
        print(f'Train Loss: {train_loss:.4f} | Train Acc: {train_acc*100:.2f}%')
        print(f'Val Loss: {val_loss:.4f} | Val Acc: {val_acc*100:.2f}%')
    
    # if val_acc > best_acc:
    #     best_acc = val_acc
    #     torch.save(model.state_dict(), 'best_model_30.pth')
    #     print('Saved best model!')

  0%|          | 0/94 [00:00<?, ?it/s]

torch.Size([512, 60]) torch.Size([512])


Epoch 1/100
Train Loss: 0.6934 | Train Acc: 50.28%
Val Loss: 0.6938 | Val Acc: 48.74%
Epoch 11/100
Train Loss: 0.6903 | Train Acc: 53.35%
Val Loss: 0.6942 | Val Acc: 50.18%


KeyboardInterrupt: 

In [ ]:
test_loss, test_acc = validate(model, train_loader, criterion, device)
print(f'Test Loss: {test_loss:.4f} | Test Acc: {test_acc*100:.2f}%')